<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [2]:
import csv
from openai import OpenAI
import time
import os
import random
import re

# OpenAI API anahtarınızı güvenli bir şekilde kullanmak için ortam değişkenlerini kullanın
client = OpenAI(
    # This is the default and can be omitted
    api_key='sk-proj-',
)

konular = [
    "Ağ Güvenliği",
    "Sensör Ağları",
    "Akıllı Ev Sistemleri",
    "İnsan-Makine Etkileşimi",
    "Veri Görselleştirme",
    "İş Süreçleri Yönetimi",
    "Bulanık Mantık Uygulamaları",
    "Karar Destek Sistemleri",
    "Bilgi Yönetimi",
    "Ürün Yaşam Döngüsü Yönetimi",
    "Çevik Metodolojiler",
    "Scrum",
    "Kanban",
    "Veri Madenciliği",
    "Güvenlik Analizi",
    "Sistem Dinamiği",
    "Süreç İyileştirme",
    "İşlevsel Güvenlik",
    "Süreç Otomasyonu",
    "Standartlar ve Sertifikasyon",
    "Etik ve Yasal Düzenlemeler",
    "Yenilik Yönetimi",
    "Değişim Yönetimi",
    "İnovasyon",
    "Teknoloji Yönetimi",
    "Performans Yönetimi",
    "Veri Güvenliği",
    "Veri Entegrasyonu",
    "Sistem Güvenliği",
    "Yazılım Entegrasyonu",
    "Donanım Entegrasyonu",
    "Sistem Arayüzleri",
    "İş Analizi",
    "Bilgi Sistemleri",
    "Optimizasyon Teknikleri"
]

soru_kaliplari = [
    "{0} nedir ve nasıl çalışır?",
    "{0} nasıl uygulanır?",
    "{0} sürecinin adımları nelerdir?",
    "{0} sırasında karşılaşılan zorluklar nelerdir?",
    "{0} için en iyi uygulamalar nelerdir?",
    "{0} neden önemlidir?",
    "{0} performansı nasıl ölçülür?",
    "{0} nasıl optimize edilir?",
    "{0} ve {1} arasındaki farklar nelerdir?",
    "{0} ve {1} entegrasyonu nasıl sağlanır?",
    "{0} uygulamalarında dikkat edilmesi gerekenler nelerdir?",
    "{0} kullanmanın avantajları ve dezavantajları nelerdir?",
    "{0} gelecekte nasıl evrilecektir?",
    "{0} için hangi araçlar kullanılabilir?",
    "{0} projelerinde risk yönetimi nasıl yapılır?",
    "{0} ile ilgili güncel trendler nelerdir?",
    "{0} sırasında hangi metrikler izlenir?",
    "{0} süreçlerinde kalite nasıl sağlanır?",
    "{0} güvenliği nasıl sağlanır?",
    "{0} uygulamalarında karşılaşılan sorunlar ve çözümleri nelerdir?",
    "{0} maliyetleri nasıl azaltılır?",
    "{0} süreçlerinde hangi standartlar kullanılır?",
    "{0} için hangi eğitimler gereklidir?",
    "{0} konusunda uzmanlaşmak için neler yapılmalıdır?",
    "{0} projelerinde takım çalışması nasıl organize edilir?",
    "{0} ve {1} arasındaki ilişki nedir?",
    "{0} için hangi test yöntemleri kullanılır?",
    "{0} projelerinde başarının anahtar faktörleri nelerdir?",
    "{0} nasıl belgelendirilir?",
    "{0} uygulamalarında etik konular nelerdir?",
    "{0} verimliliği nasıl artırılır?",
    "{0} projelerinde başarısızlık nedenleri nelerdir?",
    "{0} süreçlerinde hangi metodolojiler kullanılır?",
    "{0} süreçlerinde iletişim nasıl yönetilir?",
    "{0} ve {1} birlikte nasıl kullanılır?",
    "{0} için hangi yazılım araçları önerilir?",
    "{0} uygulamalarında en sık yapılan hatalar nelerdir?",
    "{0} süreçlerinde insan faktörü nasıl yönetilir?",
    "{0} projelerinde maliyet-fayda analizi nasıl yapılır?",
    "{0} ve {1} entegrasyonunda hangi zorluklar vardır?",
    "{0} uygulamalarında hangi veri türleri kullanılır?",
    "{0} süreçlerinde veri güvenliği nasıl sağlanır?",
    "{0} ve {1} arasındaki sinerji nasıl oluşturulur?",
    "{0} projelerinde zaman yönetimi nasıl yapılır?",
    "{0} için hangi sertifikasyon programları mevcuttur?",
    "{0} süreçlerinde değişim yönetimi nasıl uygulanır?",
    "{0} uygulamalarında müşteri gereksinimleri nasıl yönetilir?",
    "{0} projelerinde başarının ölçütleri nelerdir?",
    "{0} ve {1} teknolojileri nasıl bir arada kullanılabilir?",
    "{0} için gelecekteki teknolojik trendler nelerdir?",
    "{0} uygulamalarında hangi güvenlik önlemleri alınmalıdır?",
    "{0} süreçlerinde belgelendirme neden önemlidir?",
    "{0} projelerinde paydaş yönetimi nasıl yapılır?",
    "{0} uygulamalarında sürdürülebilirlik nasıl sağlanır?",
    "{0} süreçlerinde sürekli iyileştirme nasıl yapılır?",
    "{0} için hangi simülasyon araçları kullanılabilir?",
    "{0} projelerinde kaynak yönetimi nasıl optimize edilir?",
    "{0} ve {1} entegrasyonunun faydaları nelerdir?",
    "{0} süreçlerinde hangi metrikler izlenmelidir?",
    "{0} uygulamalarında inovasyon nasıl teşvik edilir?",
    "{0} projelerinde risk azaltma stratejileri nelerdir?",
    "{0} süreçlerinde kalite kontrol nasıl yapılır?",
    "{0} uygulamalarında büyük veri nasıl kullanılır?",
    "{0} projelerinde iletişim planlaması nasıl yapılır?",
    "{0} süreçlerinde karar verme nasıl optimize edilir?",
    "{0} ve {1} teknolojilerinin birleşimi hangi yeni imkanlar sunar?",
    "{0} uygulamalarında yapay zeka nasıl entegre edilir?",
    "{0} süreçlerinde uyumluluk nasıl sağlanır?",
    "{0} projelerinde tedarik zinciri yönetimi nasıl yapılır?",
    "{0} uygulamalarında kullanıcı deneyimi nasıl iyileştirilir?",
    "{0} süreçlerinde süreç iyileştirme nasıl yapılır?",
    "{0} projelerinde stratejik planlama nasıl yapılır?",
    "{0} uygulamalarında veri analitiği nasıl kullanılır?",
    "{0} süreçlerinde liderlik nasıl rol oynar?",
    "{0} projelerinde inovasyon stratejileri nelerdir?",
    "{0} ve {1} teknolojilerinin gelecekteki rolü nedir?",
    "{0} uygulamalarında siber güvenlik nasıl sağlanır?",
    "{0} süreçlerinde esneklik nasıl artırılır?",
    "{0} projelerinde kültürel faktörler nasıl yönetilir?",
    "{0} uygulamalarında blok zinciri teknolojisi nasıl kullanılır?",
    "{0} süreçlerinde belirsizlik nasıl yönetilir?",
    "{0} projelerinde çevik yönetim yaklaşımları nasıl uygulanır?",
    "{0} uygulamalarında makine öğrenmesi nasıl entegre edilir?",
    "{0} süreçlerinde süreç analizi nasıl yapılır?",
    "{0} projelerinde etik ve yasal sorumluluklar nelerdir?",
    "{0} uygulamalarında kullanıcı geri bildirimleri nasıl değerlendirilir?",
    "{0} süreçlerinde standardizasyon neden önemlidir?",
    "{0} projelerinde performans ölçütleri nelerdir?",
    "{0} uygulamalarında bulut teknolojileri nasıl kullanılır?",
    "{0} süreçlerinde kalite iyileştirme nasıl sağlanır?",
    "{0} projelerinde teknoloji seçimi nasıl yapılır?",
    "{0} uygulamalarında robotik süreç otomasyonu nasıl entegre edilir?",
    "{0} süreçlerinde insan kaynakları yönetimi nasıl yapılır?",
    "{0} projelerinde yenilik yönetimi nasıl uygulanır?",
    "{0} uygulamalarında nesnelerin interneti nasıl kullanılır?",
    "{0} süreçlerinde verimlilik nasıl artırılır?",
    "{0} projelerinde uluslararası standartlar nasıl uygulanır?",
    "{0} uygulamalarında veri gizliliği nasıl korunur?",
    "{0} süreçlerinde tedarikçi ilişkileri nasıl yönetilir?",
    "{0} projelerinde zaman ve maliyet tahminleri nasıl yapılır?"
]


def generate_answer(question):
    try:
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": "Sen bir mühendislik uzmanısın ve sorulara detaylı ve anlaşılır cevaplar veriyorsun. Maksimum 500 token cevap verebilirsin. Bu kapsamda cümleleri yarım bıracak şekilde cevap verme."},
                {"role": "user", "content": question}
            ],
            max_tokens=500,
            n=1,
            stop=None,
            temperature=0.7,
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Bir hata oluştu: {e}")
        return ""

# Cümleler arasındaki boşlukları kaldıran fonksiyon
def remove_spaces_between_sentences(text):
    # Nokta, ünlem veya soru işaretinden sonra gelen boşlukları kaldırır
    return re.sub(r'(?<=[.!?])\s+:', '', text)

with open('dataset.csv', 'w', encoding='utf-8-sig', newline='') as csvfile:
    fieldnames = ['instruction', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for konu1 in konular:
        for soru_kalibi in soru_kaliplari:
            if '{1}' in soru_kalibi:
                # İkinci bir konu seçiyoruz
                konu2 = random.choice(konular)
                while konu2 == konu1:
                    konu2 = random.choice(konular)
                instruction = soru_kalibi.format(konu1, konu2)
            else:
                instruction = soru_kalibi.format(konu1)
            output = generate_answer(instruction)
            # Cevabı işliyoruz
            output = remove_spaces_between_sentences(output)
            writer.writerow({'instruction': instruction, 'input': '', 'output': output})
            time.sleep(1)  # API hız sınırlarını aşmamak için bekleme süresi ekleyin

print("Veri seti oluşturuldu.")


KeyboardInterrupt: 

In [ ]:
    "Sistem Gereksinim Analizi",
    "Sistem Tasarımı",
    "Sistem Mimarisi",
    "Sistem Entegrasyonu",
    "Sistem Doğrulama ve Geçerleme",
    "Risk Yönetimi",
    "Yapılandırma Yönetimi",
    "Sistem Yaşam Döngüsü",
    "Modelleme ve Simülasyon",
    "Sistem Güvenilirliği",
    "Kalite Yönetimi",
    "Proje Yönetimi",
    "Sistem Testi",
    "İnsan Faktörleri",
    "Sistem Güvenliği",
    "Bakım ve Destek",
    "Sistem Optimizasyonu",
    "Sistem Metodolojileri",
    "Sistem Standartları",
    "Sistem Analizi",
    "Sistem Araçları",
    "Gereksinim Yönetimi",
    "Performans Analizi",
    "Maliyet Analizi",
    "Sistem Planlaması",
    "Sistem Süreçleri",
    "Sistem Yönetimi",
    "Sistem Eğitimi",
    "Sistem Uygulamaları",
    "Sistem ve Yazılım Entegrasyonu",
    "Sistem ve Donanım Entegrasyonu",
    "Ağ Sistemleri",
    "Güvenlik Sistemleri",
    "Veri Analitiği",
    "Yapay Zeka Uygulamaları",
    "Makine Öğrenmesi Uygulamaları",
    "Siber Güvenlik",
    "Bulut Bilişim",
    "Nesnelerin İnterneti",
    "Büyük Veri Yönetimi",
    "Robotik Sistemler",
    "Otomasyon Sistemleri",
    "Enerji Sistemleri",
    "Savunma Sistemleri",
    "Uzay Sistemleri",
    "Ulaşım Sistemleri",
    "Sağlık Sistemleri",
    "Telekomünikasyon Sistemleri",
    "Endüstri 4.0",
    "Akıllı Şebekeler",
    "Oyun Sistemleri",
    "Otomotiv Sistemleri",
    "Havacılık Sistemleri",
    "Denizcilik Sistemleri",
    "Tarım Teknolojileri",
    "Eğitim Teknolojileri",
    "Finansal Sistemler",
    "Lojistik Sistemleri",
    "Üretim Sistemleri",
    "Kalite Kontrol Sistemleri",
    "Çevresel Sistemler",
    "Yenilenebilir Enerji Sistemleri",
    "Veri Tabanı Sistemleri",
    "E-ticaret Sistemleri",
    "Mobil Sistemler",

In [ ]:
len(soru_kaliplari)

214